### How many games does a team needs to win to be crowned champions?

First we will need to import the standing tables

In [1]:
import re
import numpy as np
import pandas as pd
from library import (
    get_squad_as_index,
    remove_unnamed_cols,
    clean_opp_df,
    clean_main_df,
    merge_dfs,
    get_single_season_league_data,
)


In [2]:
my_list_first =pd.read_html("https://fbref.com/en/comps/9/2021-2022/")
leagues_list = my_list_first.copy()

In [3]:
shooting_dfs = merge_dfs(leagues_list[8], leagues_list[9])
shooting_dfs.head(5)

Standard                                                      ...  \
                 Gls   Sh  SoT  SoT%  Sh/90 SoT/90  G/Sh G/SoT  Dist  FK  ...   
Arsenal           60  581  186  32.0  15.29   4.89  0.09  0.30  17.8  22  ...   
Aston Villa       50  461  159  34.5  12.13   4.18  0.10  0.30  17.6  20  ...   
Brentford         46  436  141  32.3  11.47   3.71  0.09  0.28  16.3  11  ...   
Brighton          40  482  141  29.3  12.68   3.71  0.07  0.26  17.7  15  ...   
Burnley           32  405  119  29.4  10.66   3.13  0.08  0.26  17.2  14  ...   

                                                       Expected           \
            G/SoT_Opp Dist_Opp FK_Opp PK_Opp PKatt_Opp   xG_Opp npxG_Opp   
Arsenal          0.30     17.6   16.0    5.0       6.0     51.2     46.5   
Aston Villa      0.33     17.4   22.0    4.0       6.0     54.4     49.6   
Brentford        0.28     18.2   22.0    5.0       5.0     54.0     50.0   
Brighton         0.29     18.6   21.0    3.0       4.0     49.8     46.9   
Burnley          0.27     17.7   16.0    5.0       5.0     63.8     59.9   

                                              
            npxG/Sh_Opp G-xG_Opp np:G-xG_Opp  
Arsenal            0.11     -4.2        -4.5  
Aston Villa        0.11     -2.4        -1.6  
Brentford          0.10      0.0        -1.0  
Brighton           0.10     -5.8        -5.9  
Burnley            0.10    -12.8       -13.9  

[5 rows x 34 columns]

In [4]:
passing_df = merge_dfs(leagues_list[10], leagues_list[11])
passing_df.head(5)

Total                              Short             Medium  \
               Cmp    Att  Cmp% TotDist PrgDist   Cmp   Att  Cmp%    Cmp   
Arsenal      16176  19844  81.5  280868   91115  7325  8210  89.2   7187   
Aston Villa  12748  16499  77.3  220973   85417  5916  6803  87.0   5184   
Brentford    11676  16272  71.8  211867   85979  5201  6312  82.4   4797   
Brighton     16410  20588  79.7  288827   95329  7195  8172  88.0   7130   
Burnley       9453  14107  67.0  178782   80196  4296  5327  80.6   3306   

                   ...     Long Details                                  \
              Att  ... Cmp%_Opp Ast_Opp xAG_Opp xA_Opp A-xAG_Opp KP_Opp   
Arsenal      8140  ...     50.4    33.0    37.3   27.7      -4.3  316.0   
Aston Villa  6136  ...     58.0    35.0    37.7   30.1      -2.7  348.0   
Brentford    6120  ...     48.7    41.0    38.8   32.3       2.2  359.0   
Brighton     8208  ...     49.9    34.0    34.9   25.7      -0.9  342.0   
Burnley      4472  ...     50.7    36.0    46.1   35.7     -10.1  459.0   

                                                
            1/3_Opp PPA_Opp CrsPA_Opp Prog_Opp  
Arsenal       996.0   255.0      71.0    838.0  
Aston Villa  1094.0   276.0      61.0   1030.0  
Brentford    1331.0   339.0     101.0   1030.0  
Brighton      996.0   255.0      58.0    886.0  
Burnley      1220.0   407.0      83.0   1213.0  

[5 rows x 46 columns]

In [14]:
passing_df.Details.columns

Index(['Ast', 'xAG', 'xA', 'A-xAG', 'KP', '1/3', 'PPA', 'CrsPA', 'Prog',
       'Ast_Opp', 'xAG_Opp', 'xA_Opp', 'A-xAG_Opp', 'KP_Opp', '1/3_Opp',
       'PPA_Opp', 'CrsPA_Opp', 'Prog_Opp'],
      dtype='object')

In [6]:
"""
Squad Passing
- Total ->
    :Cmp: Passes Completed
    :Att: Passes Attempted
    :Cmp%: Completion Percentage
    :TotDist: Total distance in Yards that completed passes have traveled in any direction
    :PrgDist: Progressive distance: Total distance, in yards, that completed passes have 
              traveled towards the opponent's goal. 
              Note: Passes away from opponent's goal are counted as zero progressive yards.
- Short Passes between 5-15 yards->
    :Cmp: Short Passes Completed
    :Att: Passes Attempted
    :Cmp%: Completion Percentage
- Medium  Passes between 15 - 30 yards->
    :Cmp: Short Passes Completed
    :Att: Passes Attempted
    :Cmp%: Completion Percentage
- Long Passes > 30 yards->
    :Cmp: Short Passes Completed
    :Att: Passes Attempted
    :Cmp%: Completion Percentage
- Details ->
    :Ast: Assists
    :xAG: Expected Assisted Goals
    :xA: The likelihood each completed pass becomes a goal assists given 
         the pass type, phase of play, location and distance. 
    :A-xAG: Assists - Expected Assisted Goals
    :KP: Key Passes: Passes that directly lead to a shot (assisted shots)
    :1/3: Completed passes that enter the 1/3 of the pitch closest to the goal
    :PPA: Completed passes into the 18-yard box (not including set pieces)
    :CrsPA: Completed crosses into the 18-yard box (not including set pieces)
    :Prog: Progressive Passes: Completed passes that move the ball towards the 
           opponent's goal at least 10 yards from its furthest point in the last 
           six passes, or any completed pass into the penalty area. 
           Excludes passes from the defending 40% of the pitch
"""
passing_df.columns

MultiIndex([(  'Total',         'Cmp'),
            (  'Total',         'Att'),
            (  'Total',        'Cmp%'),
            (  'Total',     'TotDist'),
            (  'Total',     'PrgDist'),
            (  'Short',         'Cmp'),
            (  'Short',         'Att'),
            (  'Short',        'Cmp%'),
            ( 'Medium',         'Cmp'),
            ( 'Medium',         'Att'),
            ( 'Medium',        'Cmp%'),
            (   'Long',         'Cmp'),
            (   'Long',         'Att'),
            (   'Long',        'Cmp%'),
            ('Details',         'Ast'),
            ('Details',         'xAG'),
            ('Details',          'xA'),
            ('Details',       'A-xAG'),
            ('Details',          'KP'),
            ('Details',         '1/3'),
            ('Details',         'PPA'),
            ('Details',       'CrsPA'),
            ('Details',        'Prog'),
            (  'Total',     'Cmp_Opp'),
            (  'Total',     'Att_Opp'),


In [7]:
leagues_list[10].columns

MultiIndex([( 'Unnamed: 0_level_0',   'Squad'),
            ( 'Unnamed: 1_level_0',    '# Pl'),
            ( 'Unnamed: 2_level_0',     '90s'),
            (              'Total',     'Cmp'),
            (              'Total',     'Att'),
            (              'Total',    'Cmp%'),
            (              'Total', 'TotDist'),
            (              'Total', 'PrgDist'),
            (              'Short',     'Cmp'),
            (              'Short',     'Att'),
            (              'Short',    'Cmp%'),
            (             'Medium',     'Cmp'),
            (             'Medium',     'Att'),
            (             'Medium',    'Cmp%'),
            (               'Long',     'Cmp'),
            (               'Long',     'Att'),
            (               'Long',    'Cmp%'),
            ('Unnamed: 17_level_0',     'Ast'),
            ('Unnamed: 18_level_0',     'xAG'),
            ('Unnamed: 19_level_0',      'xA'),
            ('Unnamed: 20_level_0',   'A

In [8]:
for df in leagues_list:
    unamed = [
            col
            for col in df.columns.get_level_values(0).unique()
            if col.startswith("Unnamed:")
    ]
    
    print(f"{'=='*20}\n{unamed}\n{'=='*20}")


[]
['Unnamed: 0_level_0', 'Unnamed: 1_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Unnamed: 3_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Unnamed: 3_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Unnamed: 17_level_0', 'Unnamed: 18_level_0', 'Unnamed: 19_level_0', 'Unnamed: 20_level_0', 'Unnamed: 21_level_0', 'Unnamed: 22_level_0', 'Unnamed: 23_level_0', 'Unnamed: 24_level_0', 'Unnamed: 25_level_0']
['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Unnamed: 17_level_0', 'Unnamed: 18_level_0', 'Unnamed: 19_level_0', 'Unnam

In [9]:
df.columns

MultiIndex([('Unnamed: 0_level_0', 'Squad'),
            ('Unnamed: 1_level_0',  '# Pl'),
            ('Unnamed: 2_level_0',   '90s'),
            (       'Performance',  'CrdY'),
            (       'Performance',  'CrdR'),
            (       'Performance', '2CrdY'),
            (       'Performance',   'Fls'),
            (       'Performance',   'Fld'),
            (       'Performance',   'Off'),
            (       'Performance',   'Crs'),
            (       'Performance',   'Int'),
            (       'Performance',  'TklW'),
            (       'Performance', 'PKwon'),
            (       'Performance', 'PKcon'),
            (       'Performance',    'OG'),
            (       'Performance', 'Recov'),
            (      'Aerial Duels',   'Won'),
            (      'Aerial Duels',  'Lost'),
            (      'Aerial Duels',  'Won%')],
           )

In [10]:
df = leagues_list[10].copy()

to_be_removed = [
    "Unnamed: 0_level_0",
    "Unnamed: 1_level_0",
    "Unnamed: 2_level_0",
    "Unnamed: 3_level_0",
]
remove_those = "|".join(to_be_removed)

cols = [
    col
    for col in df.columns.get_level_values(0).unique()
    if col not in to_be_removed
]
df = df[cols]
cols_to_rename = [col for col in df.columns.get_level_values(0).unique() if col.startswith('Unnamed:')]

# Rename the columns
df = df.rename(columns={col: "Details" for col in cols_to_rename})

In [15]:
squad_passing_df = merge_dfs(leagues_list[10], leagues_list[11])

In [19]:
squad_passing_df.head(5)

Total                              Short             Medium  \
               Cmp    Att  Cmp% TotDist PrgDist   Cmp   Att  Cmp%    Cmp   
Arsenal      16176  19844  81.5  280868   91115  7325  8210  89.2   7187   
Aston Villa  12748  16499  77.3  220973   85417  5916  6803  87.0   5184   
Brentford    11676  16272  71.8  211867   85979  5201  6312  82.4   4797   
Brighton     16410  20588  79.7  288827   95329  7195  8172  88.0   7130   
Burnley       9453  14107  67.0  178782   80196  4296  5327  80.6   3306   

                   ...     Long Details                                  \
              Att  ... Cmp%_Opp Ast_Opp xAG_Opp xA_Opp A-xAG_Opp KP_Opp   
Arsenal      8140  ...     50.4    33.0    37.3   27.7      -4.3  316.0   
Aston Villa  6136  ...     58.0    35.0    37.7   30.1      -2.7  348.0   
Brentford    6120  ...     48.7    41.0    38.8   32.3       2.2  359.0   
Brighton     8208  ...     49.9    34.0    34.9   25.7      -0.9  342.0   
Burnley      4472  ...     50.7    36.0    46.1   35.7     -10.1  459.0   

                                                
            1/3_Opp PPA_Opp CrsPA_Opp Prog_Opp  
Arsenal       996.0   255.0      71.0    838.0  
Aston Villa  1094.0   276.0      61.0   1030.0  
Brentford    1331.0   339.0     101.0   1030.0  
Brighton      996.0   255.0      58.0    886.0  
Burnley      1220.0   407.0      83.0   1213.0  

[5 rows x 46 columns]

In [26]:
squad_passing_df.Details.head()

,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,Prog,Ast_Opp,xAG_Opp,xA_Opp,A-xAG_Opp,KP_Opp,1/3_Opp,PPA_Opp,CrsPA_Opp,Prog_Opp
Arsenal,41,46.4,34.6,-5.4,437,1354,329,74,1080,33.0,37.3,27.7,-4.3,316.0,996.0,255.0,71.0,838.0
Aston Villa,42,37.9,30.0,4.1,356,1019,293,70,936,35.0,37.7,30.1,-2.7,348.0,1094.0,276.0,61.0,1030.0
Brentford,33,36.6,33.8,-3.6,311,942,262,81,896,41.0,38.8,32.3,2.2,359.0,1331.0,339.0,101.0,1030.0
Brighton,30,37.3,31.5,-7.3,356,1187,342,72,1064,34.0,34.9,25.7,-0.9,342.0,996.0,255.0,58.0,886.0
Burnley,26,34.1,28.0,-8.1,286,877,240,84,729,36.0,46.1,35.7,-10.1,459.0,1220.0,407.0,83.0,1213.0


In [29]:
squad_passing_df.rename(columns={"Details": "Advanced_Passing"}, inplace=True)

In [31]:
squad_passing_df.Advanced_Passing

,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,Prog,Ast_Opp,xAG_Opp,xA_Opp,A-xAG_Opp,KP_Opp,1/3_Opp,PPA_Opp,CrsPA_Opp,Prog_Opp
Arsenal,41,46.4,34.6,-5.4,437,1354,329,74,1080,33.0,37.3,27.7,-4.3,316.0,996.0,255.0,71.0,838.0
Aston Villa,42,37.9,30.0,4.1,356,1019,293,70,936,35.0,37.7,30.1,-2.7,348.0,1094.0,276.0,61.0,1030.0
Brentford,33,36.6,33.8,-3.6,311,942,262,81,896,41.0,38.8,32.3,2.2,359.0,1331.0,339.0,101.0,1030.0
Brighton,30,37.3,31.5,-7.3,356,1187,342,72,1064,34.0,34.9,25.7,-0.9,342.0,996.0,255.0,58.0,886.0
Burnley,26,34.1,28.0,-8.1,286,877,240,84,729,36.0,46.1,35.7,-10.1,459.0,1220.0,407.0,83.0,1213.0
Chelsea,52,51.5,42.6,0.5,439,1614,437,66,1345,20.0,25.6,21.0,-5.6,232.0,830.0,209.0,60.0,732.0
Crystal Palace,31,36.7,30.2,-5.7,305,990,270,75,912,30.0,31.9,29.8,-1.9,304.0,1193.0,287.0,74.0,1012.0
Everton,29,33.5,25.1,-4.5,323,897,255,77,802,46.0,45.8,36.2,0.2,372.0,1448.0,329.0,74.0,1238.0
Leeds United,26,35.8,25.1,-9.8,347,995,272,53,1105,59.0,52.8,38.0,6.2,436.0,1003.0,309.0,58.0,1032.0
Leicester City,46,37.5,26.9,8.5,312,1025,254,57,1008,36.0,47.1,38.1,-11.1,415.0,1210.0,342.0,77.0,1099.0
